In [50]:
import pandas as pd
import numpy as np
from datetime import datetime
import os

In [81]:
def obtener_fecha(df):
    df['fecha'] = pd.to_datetime(df['validTimeLocal'], utc=True)
    df = df.sort_values(by=['fecha'])
    return df

In [69]:
def descomponer_fecha(df):
    df['hora'] = df['fecha'].dt.hour
    df['dia'] = df['fecha'].dt.day
    df['mes'] = df['fecha'].dt.month
    df['anio'] = df['fecha'].dt.year
    return df

In [82]:
def obtener_columnas_relevantes(df):
    df = df.rename(columns={'temperature': 'temp', 
                            'temperatureDewPoint': 'dewPt', 
                            'temperatureHeatIndex':'heat_index', 
                            'relativeHumidity':'rh', 
                            'pressureMeanSeaLevel':'pressure', 
                            'visibility':'vis', 
                            'temperatureWindChill':'wc', 
                            'temperatureFeelsLike':'feels_like',
                            'uvIndex':'uv_index',
                            'validTimeUtc': 'valid_time_gmt'})
    df = df[['hora', 'dia', 'mes', 'anio', 'temp', 'dewPt', 'heat_index', 'rh', 'pressure', 'vis', 'wc', 'feels_like', 'uv_index', 'valid_time_gmt', 'windDirectionCardinal']]
    return df

In [83]:
data = pd.read_json(r"D:\Users\Mario\Downloads\AA\15day_14_02052024_v2.json")
data

,cloudCover,dayOfWeek,dayOrNight,expirationTimeUtc,iconCode,iconCodeExtend,precipChance,precipType,pressureMeanSeaLevel,qpf,...,validTimeLocal,validTimeUtc,visibility,windDirection,windDirectionCardinal,windGust,windSpeed,wxPhraseLong,wxPhraseShort,wxSeverity
0,100,Thursday,D,1714650508,12,1200,92,rain,29.67,0.04,...,2024-05-02T13:00:00+0100,1714651200,5.0,301,WNW,18.0,11,Rain,Rain,1
1,100,Thursday,D,1714650508,12,1200,88,rain,29.67,0.05,...,2024-05-02T14:00:00+0100,1714654800,5.0,286,WNW,17.0,10,Rain,Rain,1
2,100,Thursday,D,1714650508,12,1200,76,rain,29.67,0.02,...,2024-05-02T15:00:00+0100,1714658400,5.0,278,W,NaN,10,Rain,Rain,1
3,100,Thursday,D,1714650508,12,1200,70,rain,29.67,0.02,...,2024-05-02T16:00:00+0100,1714662000,4.0,275,W,17.0,11,Rain,Rain,1
4,97,Thursday,D,1714650508,11,1100,47,rain,29.67,0.02,...,2024-05-02T17:00:00+0100,1714665600,5.0,279,W,17.0,10,Showers,Showers,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,67,Friday,D,1714651166,11,1100,43,rain,29.91,0.00,...,2024-05-17T08:00:00+0100,1715929200,7.0,275,W,NaN,9,Showers,Showers,1
356,68,Friday,D,1714651166,11,1100,42,rain,29.91,0.00,...,2024-05-17T09:00:00+0100,1715932800,7.0,265,W,NaN,9,Showers,Showers,1
357,70,Friday,D,1714651166,11,1100,41,rain,29.91,0.01,...,2024-05-17T10:00:00+0100,1715936400,7.0,259,W,NaN,10,Showers,Showers,1
358,71,Friday,D,1714651166,11,1100,39,rain,29.91,0.01,...,2024-05-17T11:00:00+0100,1715940000,7.0,228,SW,NaN,10,Showers,Showers,1


In [84]:
data = obtener_fecha(data)
data[['fecha']]

,fecha
0,2024-05-02 12:00:00+00:00
1,2024-05-02 13:00:00+00:00
2,2024-05-02 14:00:00+00:00
3,2024-05-02 15:00:00+00:00
4,2024-05-02 16:00:00+00:00
...,...
355,2024-05-17 07:00:00+00:00
356,2024-05-17 08:00:00+00:00
357,2024-05-17 09:00:00+00:00
358,2024-05-17 10:00:00+00:00


In [85]:
data = descomponer_fecha(data)
data[['hora', 'dia', 'mes', 'anio']]

,hora,dia,mes,anio
0,12,2,5,2024
1,13,2,5,2024
2,14,2,5,2024
3,15,2,5,2024
4,16,2,5,2024
...,...,...,...,...
355,7,17,5,2024
356,8,17,5,2024
357,9,17,5,2024
358,10,17,5,2024


In [86]:
data = obtener_columnas_relevantes(data)
data

,hora,dia,mes,anio,temp,dewPt,heat_index,rh,pressure,vis,wc,feels_like,uv_index,valid_time_gmt,windDirectionCardinal
0,12,2,5,2024,49,48,49,97,29.67,5.0,44,44,1,1714651200,WNW
1,13,2,5,2024,50,49,50,98,29.67,5.0,45,45,1,1714654800,WNW
2,14,2,5,2024,50,49,50,98,29.67,5.0,46,46,2,1714658400,W
3,15,2,5,2024,51,49,51,96,29.67,4.0,46,46,1,1714662000,W
4,16,2,5,2024,51,49,51,93,29.67,5.0,47,47,1,1714665600,W
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,7,17,5,2024,52,48,52,87,29.91,7.0,48,48,1,1715929200,W
356,8,17,5,2024,53,49,53,86,29.91,7.0,49,49,1,1715932800,W
357,9,17,5,2024,54,49,54,83,29.91,7.0,51,51,2,1715936400,W
358,10,17,5,2024,55,49,55,80,29.91,7.0,52,52,3,1715940000,SW


In [14]:
data.to_json(r"D:\Users\Mario\Downloads\AA\8.json")

In [15]:
data = pd.read_json(r"D:\Users\Mario\Downloads\AA\8.json")
data

,temp,dewPt,heat_index,rh,pressure,vis,wc,feels_like,uv_index,valid_time_gmt,windDirectionCardinal
0,49,48,49,97,29.67,5.0,44,44,1,1714651200,WNW
1,50,49,50,98,29.67,5.0,45,45,1,1714654800,WNW
2,50,49,50,98,29.67,5.0,46,46,2,1714658400,W
3,51,49,51,96,29.67,4.0,46,46,1,1714662000,W
4,51,49,51,93,29.67,5.0,47,47,1,1714665600,W
...,...,...,...,...,...,...,...,...,...,...,...
355,52,48,52,87,29.91,7.0,48,48,1,1715929200,W
356,53,49,53,86,29.91,7.0,49,49,1,1715932800,W
357,54,49,54,83,29.91,7.0,51,51,2,1715936400,W
358,55,49,55,80,29.91,7.0,52,52,3,1715940000,SW
